In [ ]:
import covid
from datetime import datetime,timedelta, date
from IPython.display import clear_output
from IPython.display import Markdown
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd


In [ ]:
%%html
<style>
.p-Widget.js-plotly-plot {
    height: 30vw;
    width: 70vw;
}
</style>

In [ ]:
# Config

log_scale = False
renderer = 'notebook_connected'
pio.renderers.default = renderer
plotly_conf = {'displayModeBar': True, 'responsive': True}
plt_template = 'simple_white+gridon'

covid.load_regional()
covid.set_region('Lombardia')
covid.update_data()

data = covid.data
updated_at = covid.updated_at


# Dati Coronavirus Lombardia

In [ ]:

def format_row_wise(styler, formatter):
    for row, row_formatter in formatter.items():
        row_num = styler.index.get_loc(row)
        for col_num in range(len(styler.columns)):
            styler._display_funcs[(row_num, col_num)] = row_formatter            
    return styler

def color_negative(val):    
    color = 'red' if val < 0 else 'black'
    weight = 'bold' if val < 0 else 'normal'
    return 'color: %s; font-weight:%s' % (color, weight)

styles = [
    dict(selector="*", props=[("font-size", "105%")])
]


statistics = covid.statistics
recap = covid.recap

display(Markdown("### *Aggiornamento al " + updated_at + "*"))

formatters = {
    'Mortalità': '{:+.2%}'.format,
    'Critici': '{:+.2%}'.format,
    'Ricoverati': '{:+.2%}'.format,
    'Guariti': '{:+.2%}'.format
}

recap_style = recap.style
recap_style.set_table_styles(styles)
recap_style.format('{:,.0f}'.format)
recap_style.format('{:+,.0f}'.format, subset=['Variazione tot rispetto a ieri'])
recap_style.format('{:+.2%}'.format, subset=['Variazione % rispetto a ieri'])
recap_style.applymap(color_negative)
styler = format_row_wise(recap_style, formatters)

out1 = widgets.Output()
with out1:
    display(recap_style)

out2 = widgets.Output()

pivot = covid.pivot
formatters = {
    '% mortalita': '{:,.2%}'.format,
    '% intensivi': '{:,.2%}'.format,
    '% ricoverati': '{:,.2%}'.format,
    '% guariti': '{:,.2%}'.format
}

pivot_style = pivot.iloc[:,0:7].style
pivot_style.set_table_styles(styles)
pivot_style.format('{:,.0f}'.format)
styler = format_row_wise(pivot_style, formatters)

with out2:
    display(styler)

    
tab_result = widgets.Tab(children = [out1, out2])
tab_result._dom_classes = ['rendered_html']
tab_result.set_title(0, 'Dati odierni')
tab_result.set_title(1, 'Ultimi 7 giorni')
    
display(tab_result)

## Grafici andamenti 


In [ ]:
# draw reference areas for lockdown periods

def draw_reflines(fig):
    opacity = 0.25
    line_width = 1
    line_color="Black"
    shapes = []
    for xref in range(1,4):
        shapes.extend([
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref=f"x{xref}",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=datetime(2020,3,9),
                y0=0,
                y1=1,
                x1=datetime(2020,3,21),
                fillcolor="LightSalmon",
                opacity=opacity,
                layer="below",
                line_width=line_width,             
                line_color=line_color,
                line_dash='dash'
            ),
            dict(
                type="rect",
                xref=f"x{xref}",
                yref="paper",
                x0=datetime(2020,3,21),
                y0=0,      
                y1=1,
                x1=datetime(2020,5,3),
                fillcolor="LightCoral",
                opacity=opacity,
                layer="below",
                line_width=line_width,
                line_color=line_color,
                line_dash='dash'                
            ),
            dict(
                type="rect",
                xref=f"x{xref}",
                yref="paper",
                x0=datetime(2020,5,3),
                y0=0,
                y1=1,
                x1=datetime(2020,5,17),
                fillcolor="Green",
                opacity=opacity,
                layer="below",
                line_width=line_width,
                line_color=line_color,
                line_dash='dash'                
            ),
            dict(
                type="rect",
                xref=f"x{xref}",
                yref="paper",
                x0=datetime(2020,5,17),
                y0=0,
                x1=datetime(2020,6,2),
                y1=1,
                fillcolor="LightGreen",
                opacity=opacity,
                layer="below",
                line_width=line_width,
                line_color=line_color,
                line_dash='dash'                
            )])
    fig.update_layout(shapes=shapes)


In [ ]:
# plot
def smooth(datas):
    return datas.rolling(7,
        win_type='gaussian',
        min_periods=1,
        center=True).mean(std=2).round()

line_color = ['blue','red','green']    

def draw_plotly(log_scale):
    fig = make_subplots(rows=1, cols=3,
                        shared_xaxes=False,
                        subplot_titles=("Totale positivi", "Variazione positivi", "Terapia intensiva"))
                                      
    fig.add_trace(
        go.Scatter(x=data.date, y=data.totale_positivi, line_color=line_color[0], name='totale positivi'),
        row=1, col=1
    )
    if log_scale: fig.update_yaxes(type="log", row=1, col=1)
    fig.add_trace(
        go.Scatter(x=data.date, y=data.variazione_totale_positivi, line_color='lightgreen', name='variazione positivi'),
        row=1, col=2
    )        
    fig.add_trace(
        go.Scatter(x=data.date, y=smooth(data.variazione_totale_positivi), line_color=line_color[1], 
                   line_dash='dashdot', name='media mobile variazione positivi'),
        row=1, col=2
    )

    if log_scale: fig.update_yaxes(type="log", row=1, col=2)    
    ax = fig.add_trace(
        go.Scatter(x=data.date, y=data.terapia_intensiva, line_color=line_color[2], name='terapia intensiva'),
        row=1, col=3
    )
    if log_scale: fig.update_yaxes(type="log", row=1, col=3)    

    fig.update_layout(template=plt_template, showlegend=False)
    draw_reflines(fig)
    
    return fig
        
        
def draw_plotly2(log_scale):         
    fig = make_subplots(rows=1, cols=3,
                        shared_xaxes=False,
                        subplot_titles=("Ospedalizzati","Deceduti", "Dimessi"))
    
    fig.add_trace(
        go.Scatter(x=data.date, y=data.totale_ospedalizzati, line_color=line_color[0], name='ospedalizzati'),
        row=1, col=1
    )
    if log_scale: fig.update_yaxes(type="log", row=2, col=1)    
    fig.add_trace(
        go.Scatter(x=data.date, y=data.deceduti, line_color=line_color[1], name='deceduti'),
        row=1, col=2
    )
    if log_scale: fig.update_yaxes(type="log", row=2, col=2)    
    fig.add_trace(
        go.Scatter(x=data.date, y=data.dimessi_guariti, line_color=line_color[2], name='dimessi'),
        row=1, col=3
    )
    if log_scale: fig.update_yaxes(type="log", row=2, col=3)   

    fig.update_layout(template=plt_template, showlegend=False)
    draw_reflines(fig)
    
    return fig
        
        
def draw_plotly3(log_scale):         
    fig = make_subplots(rows=1, cols=3,
                        shared_xaxes=False,
                        subplot_titles=("Nuovi decessi", "Nuovi positivi", "Tamponi"))        
    fig.add_trace(
        go.Scatter(x=data.date, y=data.nuovi_decessi, line_color='lightgreen', name='nuovi decessi'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=data.date, y=data.nuovi_decessi.rolling(7).mean(), line_color=line_color[0], 
                   line_dash='dashdot',name='media mobile nuovi decessi'),
        row=1, col=1
    )
    
    if log_scale: fig.update_yaxes(type="log", row=3, col=1)    
    fig.add_trace(
        go.Scatter(x=data.date, y=data.nuovi_positivi, line_color='lightgreen', name='nuovi positivi'),
        row=1, col=2
    )        
    fig.add_trace(
        go.Scatter(x=data.date, y=smooth(data.nuovi_positivi), line_color=line_color[1], 
                   line_dash='dashdot', name='media mobile nuovi positivi'),
        row=1, col=2
    )
    if log_scale: fig.update_yaxes(type="log", row=3, col=2)    
    fig.add_trace(
        go.Scatter(x=data.date, y=data.tamponi, line_color=line_color[2], name='tamponi'),
        row=1, col=3
    )        
    if log_scale: fig.update_yaxes(type="log", row=3, col=3)    

    fig.update_layout(template=plt_template, showlegend=False)
    draw_reflines(fig)
    
    return fig


#create tabs
log_scale = False    
out1 = draw_plotly(log_scale)
out2 = draw_plotly2(log_scale)
out3 = draw_plotly3(log_scale)
log_scale = True
out4 = draw_plotly(log_scale)
out5 = draw_plotly2(log_scale)
out6 = draw_plotly3(log_scale)

# config={'displayModeBar': False}
g1 = go.FigureWidget(data=out1,     
                     layout=go.Layout())
g2 = go.FigureWidget(data=out2,                     
                     layout=go.Layout())
g3 = go.FigureWidget(data=out3,                     
                     layout=go.Layout())
g4 = go.FigureWidget(data=out4,                     
                     layout=go.Layout())
g5 = go.FigureWidget(data=out5,                     
                     layout=go.Layout())
g6 = go.FigureWidget(data=out6,                     
                     layout=go.Layout())

container1 = widgets.VBox(children=[g1,g2,g3])
container2 = widgets.VBox(children=[g4,g5,g6])

tab_nest = widgets.Tab(children = [container1, container2])
tab_nest.set_title(0, 'Lineari')
tab_nest.set_title(1, 'Logaritmici')

display(tab_nest)


## Dettaglio andamento casi

In [ ]:
# Plotly chart
opacity = 0.2
line_width = 1
line_color="Black"

shapes = [
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=datetime(2020,3,9),
                y0=0,
                y1=1,
                x1=datetime(2020,3,21),
                fillcolor="LightSalmon",
                opacity=opacity,
                layer="below",
                line_width=line_width,             
                line_color=line_color,
                line_dash='dash'

            ),
            dict(
                type="rect",
                xref=f"x",
                yref="paper",
                x0=datetime(2020,3,21),
                y0=0,      
                y1=1,
                x1=datetime(2020,5,3),
                fillcolor="LightCoral",
                opacity=opacity,
                layer="below",
                line_width=line_width,             
                line_color=line_color,
                line_dash='dash'
                
            ),
            dict(
                type="rect",
                xref=f"x",
                yref="paper",
                x0=datetime(2020,5,3),
                y0=0,
                y1=1,
                x1=datetime(2020,5,17),
                fillcolor="Green",
                opacity=opacity,
                layer="below",
                line_width=line_width,             
                line_color=line_color,
                line_dash='dash'
            ),
            dict(
                type="rect",
                xref=f"x",
                yref="paper",
                x0=datetime(2020,5,17),
                y0=0,
                x1=datetime(2020,6,2),
                y1=1,
                fillcolor="LightGreen",
                opacity=opacity,
                layer="below",
                line_width=line_width,             
                line_color=line_color,
                line_dash='dash'
            )]
fig = px.line(data, 
              data.date, 
              [data.totale_positivi, data.totale_ospedalizzati], 
              color_discrete_map={
                 "totale_positivi": "red",
                 "totale_ospedalizzati": "blue"}
             )
fig.update_layout(template=plt_template, shapes=shapes, legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    bordercolor="Darkgrey",
    borderwidth=1,
    title=''
))
fig.show(config={'responsive':True})




## Analisi predittiva variazione nuovi positivi

In [ ]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly

delta_positivi = data[['date', 'variazione_totale_positivi']][20:]
delta_positivi = delta_positivi.rename(columns={'date':'ds', 'variazione_totale_positivi':'y'})
# remove outliers
delta_positivi = delta_positivi.drop(delta_positivi[delta_positivi.y <= -2500].index)
delta_positivi = delta_positivi.drop(delta_positivi[delta_positivi.y >= 4500].index)
# model fit
model = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
model.fit(delta_positivi)
future_days = model.make_future_dataframe(periods=60, freq='D')
# forecasting
forecast = model.predict(future_days)
fig = plot_plotly(model, forecast)
fig.update_layout(template=plt_template)
fig.show(config={'displayModeBar': True})
fig = plot_components_plotly(model, forecast)
fig.update_layout(template=plt_template, height=400)
fig.show(config={'displayModeBar': True, 'responsive': True})


In [ ]:
original, smoothed = covid.prepare_cases(data.totale_casi)
posteriors, log_likelihood = covid.get_posteriors(smoothed, sigma=.15)

In [ ]:

hdis = covid.highest_density_interval(posteriors, p=.9)
most_likely = posteriors.idxmax().rename('ML')
result = pd.concat([most_likely, hdis], axis=1)

display(Markdown(f"## Stima $R_t$=  {result['ML'].values[-1]} \
                 ({result['Low_90'].values[-1]} - {result['High_90'].values[-1]})"))

fig = go.Figure()
fig.add_trace(go.Scatter(x=result.index, y=result['ML'],  
                    mode='lines+markers',
                    marker=dict(
                            size=6,
                            color=result['ML'], #set color equal to a variable
                            showscale=True
                        ),
                    error_y=dict(
                        type='data',
                        array = result['High_90'] - result['Low_90'],
                        width = 0,
                        thickness = 0.7,
                        color = 'darkgray'
                    ), 
                    name='Rt'))
fig.update_layout(template=plt_template, yaxis_range=[-2,3])
fig.show(config={'responsive':True})



## Analisi andamento $R_t$ 

In [ ]:

rt = pd.DataFrame({'ds':data['date'].values, 'y': result['ML'].values})
# model fit
model_rt = Prophet(daily_seasonality=False, yearly_seasonality=False)
model_rt.fit(rt)
future_days = model.make_future_dataframe(periods=60, freq='D')
# forecasting
forecast = model_rt.predict(future_days)
fig = plot_components_plotly(model_rt, forecast)
fig.update_layout(template=plt_template, height=600)
fig.show(config={'displayModeBar': True, 'responsive': True})





*fonte*: [Dati DPC](https://github.com/pcm-dpc/COVID-19/blob/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv)